In [1]:
import re
import nltk
from sklearn import svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from google_trans_new import google_translator  
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
fg_df = pd.read_csv('D:/Users/ehern/Desktop/Machine Learning/Project/fall_guys_metacritic_reviews.csv')

fg_df.head()

,username,review_type,published_date,score,votes,review_text,profile_url,platform
0,Fen1ma,user,"Aug 26, 2020",6.0,0.0,"на данный момент не советую покупать игру, ещё...",https://www.metacritic.com/user/Fen1ma,pc
1,Locke_VI,user,"Aug 25, 2020",3.0,0.0,Fall Guys was a game I was really looking forw...,https://www.metacritic.com/user/Locke_VI,pc
2,Crisener,user,"Aug 25, 2020",8.0,0.0,You gotta love it. The aesthetics are very col...,https://www.metacritic.com/user/Crisener,pc
3,EnigmaXtreme,user,"Aug 25, 2020",7.0,0.0,When we covered Fall Guys last week during it’...,https://www.metacritic.com/user/EnigmaXtreme,pc
4,nitorita,user,"Aug 23, 2020",7.0,0.0,I like how Fall Guys adapted Mario Party and m...,https://www.metacritic.com/user/nitorita,pc


In [3]:
fg_df.drop(columns=['username','review_type','published_date','published_date', 'votes', 'profile_url', 'platform'],inplace=True)

In [4]:
noNull = fg_df.dropna()

In [5]:
# for i in noNull['review_text']:
#     detector = google_translator()  
#     detect_result = detector.detect(i)
#     print(detect_result)

In [6]:
translations = {}
for i in noNull['review_text']:
    translator = google_translator()
    translations[i] = translator.translate(i, lang_tgt='en')
    
noNull.replace(translations, inplace = True)

# check translation
noNull

C:\Users\ehern\anaconda3\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,score,review_text
0,6.0,At the moment I do not advise you to buy the g...
1,3.0,Fall Guys was a game I was really looking forw...
2,8.0,You gotta love it. The aesthetics are very col...
3,7.0,When we covered Fall Guys last week during it’...
4,7.0,I like how Fall Guys adapted Mario Party and m...
...,...,...
349,1.0,Fall guys is a great game that is destroyed by...
350,9.0,"Simple and fun, very easy to understand gamepl..."
351,8.0,"HIVALLY FOR ALL ages, and when you only have a..."
352,9.0,This game is fun but when you play with friend...


In [7]:
translated_df = noNull.copy()

In [8]:
translated_df.isnull().sum()

score          0
review_text    0
dtype: int64

In [9]:
translated_df.describe()

,score
count,329.000000
mean,6.802432
std,3.340512
min,0.000000
25%,5.000000
50%,8.000000
75%,10.000000
max,10.000000


In [10]:
translated_df['score'].value_counts()

10.0    83
8.0     61
9.0     50
0.0     34
7.0     26
6.0     20
5.0     15
2.0     12
1.0     11
3.0     10
4.0      7
Name: score, dtype: int64

In [11]:
translated_df['review_text'] = translated_df['review_text'].str.replace('[^\w\s]','')

In [12]:
translated_df['review_text'] = translated_df['review_text'].str.lower()

In [13]:
translated_df.head()

,score,review_text
0,6.0,at the moment i do not advise you to buy the g...
1,3.0,fall guys was a game i was really looking forw...
2,8.0,you gotta love it the aesthetics are very colo...
3,7.0,when we covered fall guys last week during its...
4,7.0,i like how fall guys adapted mario party and m...


In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
def tokenize(text):
    tokens = re.split('\W+', str(text))
    return tokens

translated_df['review_text'] = translated_df['review_text'].apply(lambda w: tokenize(w))
translated_df.head()

,score,review_text
0,6.0,"[at, the, moment, i, do, not, advise, you, to,..."
1,3.0,"[fall, guys, was, a, game, i, was, really, loo..."
2,8.0,"[you, gotta, love, it, the, aesthetics, are, v..."
3,7.0,"[when, we, covered, fall, guys, last, week, du..."
4,7.0,"[i, like, how, fall, guys, adapted, mario, par..."


In [16]:
stopwords = nltk.corpus.stopwords.words('english')
def removeStopWords(text):
    cleanText = [w for w in text if w not in stopwords]
    return cleanText

translated_df['review_text'] = translated_df['review_text'].apply(lambda w: removeStopWords(w))
translated_df.head()

,score,review_text
0,6.0,"[moment, advise, buy, game, still, raw, br, mi..."
1,3.0,"[fall, guys, game, really, looking, forward, f..."
2,8.0,"[gotta, love, aesthetics, colorful, charming, ..."
3,7.0,"[covered, fall, guys, last, week, beta, talked..."
4,7.0,"[like, fall, guys, adapted, mario, party, made..."


In [17]:
# def listToString(text): 
#     str1 = " " 
#     return (str1.join(text))
        
# translated_df['review_text'] = translated_df['review_text'].apply(lambda w: listToString(w))
# translated_df.head()

In [18]:
vectorizer = TfidfVectorizer()

In [22]:
X = translated_df['review_text']
y = translated_df['score']

In [23]:
X

0      [moment, advise, buy, game, still, raw, br, mi...
1      [fall, guys, game, really, looking, forward, f...
2      [gotta, love, aesthetics, colorful, charming, ...
3      [covered, fall, guys, last, week, beta, talked...
4      [like, fall, guys, adapted, mario, party, made...
                             ...                        
349    [fall, guys, great, game, destroyed, couple, p...
350    [simple, fun, easy, understand, gameplay, bett...
351               [hivally, ages, nothing, difficulty, ]
352    [game, fun, play, friends, becomes, really, fu...
353    [awesome, lot, fun, laugh, im, worried, enough...
Name: review_text, Length: 329, dtype: object

In [24]:
y

0      6.0
1      3.0
2      8.0
3      7.0
4      7.0
      ... 
349    1.0
350    9.0
351    8.0
352    9.0
353    9.0
Name: score, Length: 329, dtype: float64

In [25]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict)
    return accuracy

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.28, random_state=31)

print("Accuracy:", get_score(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))

cv_score_dt = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_dt.mean()))

ValueError: setting an array element with a sequence.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

print("Accuracy:", get_score(LogisticRegression(), X_train, X_test, y_train, y_test))

cv_score_lr = cross_val_score(LogisticRegression(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_lr.mean()))

In [27]:
# from sklearn.metrics import classification_report

# print(classification_report(y_test, y_predict_dt))

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=31)

print("Accuracy:", get_score(RandomForestClassifier(), X_train, X_test, y_train, y_test))

cv_score_rf = cross_val_score(RandomForestClassifier(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_rf.mean()))

ValueError: setting an array element with a sequence.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.26, random_state=31)

print("Accuracy:", get_score(svm.SVC(), X_train, X_test, y_train, y_test))

cv_score_svm = cross_val_score(svm.SVC(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_svm.mean()))

ValueError: setting an array element with a sequence.

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# kf = StratifiedKFold(n_splits = 5)